In [120]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None 

import json 
from datetime import datetime


In [ ]:
stationId

In [ ]:
https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=CWB-29B0277F-A9E5-417F-BF53-2502DC924680&format=JSON&locationName=%E9%B4%9B%E9%B4%A6%E6%B9%96,%E5%8D%97%E5%B1%B1,%E5%A4%AA%E5%B9%B3%E5%B1%B1,%E7%8E%89%E8%98%AD,%E6%80%9D%E6%BA%90&elementName=ELEV,TEMP,HUMD,H_24R,D_TX,D_TN,D_TXT,D_TNT&parameterName%EF%BC%8C=

In [ ]:
Authorization='CWB-29B0277F-A9E5-417F-BF53-2502DC924680'

In [125]:
url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001'
    
r = requests.get(url, 
                 {'Authorization':'CWB-29B0277F-A9E5-417F-BF53-2502DC924680',
                  'format':'JSON',
                  'stationId':['C0U720','C0UA10', 'C0U710', 'C0UA30']
                 })
r.encoding = 'utf8'

In [126]:
j = r.json()['records']['location']

df = pd.json_normalize(j, 'weatherElement', ['locationName', 'stationId', 'time'])
df['time'] = df['time'].apply(lambda x: x['obsTime'])

df = df.pivot_table(values='elementValue', 
               index=['locationName', 'stationId', 'time'], 
               columns='elementName', aggfunc='first')

In [127]:
df2 = df[['ELEV', 'H_24R', 'HUMD', 'TEMP', 'D_TN', 'D_TX', 'D_TNT', 'D_TXT']]
df2['D_TNT'] = df2['D_TNT'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S+08:00').time())
df2['D_TXT'] = df2['D_TXT'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S+08:00').time())

In [128]:
df2

,,elementName,ELEV,H_24R,HUMD,TEMP,D_TN,D_TX,D_TNT,D_TXT
locationName,stationId,time,,,,,,,,
南山,C0U720,2021-01-08 13:00:00,1260.0,3.0,1,0.3,-0.30,2.00,08:00:00,00:30:00
太平山,C0U710,2021-01-08 13:00:00,1942.0,0.5,1,-2.8,-4.50,-2.00,07:30:00,00:20:00
白嶺,C0UA30,2021-01-08 13:00:00,1397.0,0.5,1,-0.3,-1.10,0.90,07:50:00,00:10:00
鴛鴦湖,C0UA10,2021-01-08 13:00:00,1711.0,0.0,1,-1.8,-2.60,-0.40,09:20:00,00:20:00


In [116]:
pd.to_datetime(df['D_TNT'], format='%Y-%m-%dT%H:%M:%S+08:00')
df['D_TXT'] = pd.to_datetime(df['D_TXT'], format='%Y-%m-%dT%H:%M:%S+08:00')
df

AttributeError: 'Series' object has no attribute 'time'

In [114]:
print(df['D_TNT'][1].time)

<built-in method time of Timestamp object at 0x7fb44ca93f50>


In [92]:
df

,elementName,ELEV,H_24R,HUMD,TEMP,D_TN,D_TX,D_TNT,D_TXT
locationName,stationId,,,,,,,,
南山,C0U720,1260.0,3.0,1,0.3,-0.30,2.00,2021-01-08 08:00:00,2021-01-08 00:30:00
太平山,C0U710,1942.0,0.5,1,-2.8,-4.50,-2.00,2021-01-08 07:30:00,2021-01-08 00:20:00
白嶺,C0UA30,1397.0,0.5,1,-0.3,-1.10,0.90,2021-01-08 07:50:00,2021-01-08 00:10:00
鴛鴦湖,C0UA10,1711.0,0.0,1,-1.8,-2.60,-0.40,2021-01-08 09:20:00,2021-01-08 00:20:00


In [81]:
datetime.strptime(df['D_TNT'][0], '%Y-%m-%dT%H:%M:%S+08:00')

ValueError: unconverted data remains: +08:00